In [143]:
import pandas as pd

In [144]:
movies = pd.read_csv("/kaggle/input/movies-and-ratings/movies.csv", usecols = ['movieId', 'title'])
ratings = pd.read_csv("/kaggle/input/movies-and-ratings/ratings.csv", usecols = ['userId', 'movieId', 'rating'])
movies.head()


,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [145]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [146]:
movies.shape

(9742, 2)

In [147]:
ratings.shape

(100836, 3)

In [148]:
movie_users = ratings.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
movie_users

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors as NN

In [150]:
mat_movies = csr_matrix(movie_users.values)

Cosine similarity is a measure of similarity between two non-zero vectors in a multi-dimensional space. It calculates the cosine of the angle between the vectors, where:

Cosine Similarity
=
𝐴
⋅
𝐵
∥
𝐴
∥
∥
𝐵
∥
Cosine Similarity= 
∥A∥∥B∥
A⋅B
​
 


In [151]:
model = NN(metric = 'cosine', algorithm = 'brute', n_neighbors = 20)
model.fit(mat_movies)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [152]:
#To improve the search process
from fuzzywuzzy import process

In [153]:
def recommender(movie_name, data, n):
    idx = process.extractOne(movie_name, movies['title'])[2]
    print('Movie Selected: ', movies['title'][idx], 'Index: ', idx)
    print('Searching for recommendations........')
    distance, indices = model.kneighbors(data[idx], n_neighbors=n)
    print(distance, indices)
    for i in indices:
        print(movies['title'][i].where(i!=idx))

In [154]:
recommender('Jumanji', mat_movies, 10)

Movie Selected:  Jumanji (1995) Index:  1
Searching for recommendations........
[[0.         0.41156227 0.45018189 0.45501892 0.46195443 0.47512358
  0.48183868 0.48438002 0.49254201 0.50243974]] [[  1 322 436 325 418 504 483 506 512  18]]
1                                         NaN
322                     Lion King, The (1994)
436                     Mrs. Doubtfire (1993)
325                          Mask, The (1994)
418                      Jurassic Park (1993)
504                         Home Alone (1990)
483    Nightmare Before Christmas, The (1993)
506                            Aladdin (1992)
512               Beauty and the Beast (1991)
18      Ace Ventura: When Nature Calls (1995)
Name: title, dtype: object
